<a href="https://colab.research.google.com/github/larregui/Library/blob/main/DeskScheduleCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
from collections import defaultdict
from datetime import datetime, timedelta
import pandas as pd

# Define staff and their maximum desk hours
staff = {
    "Laura": {"role": "Librarian II", "max_hours": 2},
    "Phil": {"role": "Librarian I", "max_hours": 4},
    "Keely": {"role": "Librarian I", "max_hours": 4},
    "Kerry": {"role": "Paraprofessional", "max_hours": 4},
    "Cesar": {"role": "Paraprofessional", "max_hours": 4},
    "Deepna": {"role": "Paraprofessional", "max_hours": 4},
    "Sony": {"role": "Library Assistant", "max_hours": 6},
    "Gabby": {"role": "Library Assistant", "max_hours": 6},
    "Luna": {"role": "Library Assistant", "max_hours": 6},
    "Jaryn": {"role": "Library Assistant", "max_hours": 6}
}

# Define specific working hours for each staff member
working_hours = {
    "Laura": {
        "Monday": ("12:00", "21:00"),
        "Wednesday": ("08:00", "17:00"),
        "Thursday": ("08:00", "17:00"),
        "Friday": ("08:00", "17:00")
    },
    "Phil": {
        "Monday": ("12:00", "21:00"),
        "Tuesday": ("09:00", "18:00"),
        "Wednesday": ("08:00", "17:00"),
        "Thursday": ("08:00", "17:00"),
        "Friday": ("08:00", "17:00")
    },
    "Keely": {
        "Monday": ("08:00", "17:00"),
        "Tuesday": ("08:00", "17:00"),
        "Wednesday": ("09:00", "18:00"),
        "Thursday": ("12:00", "21:00"),
        "Friday": ("08:00", "17:00")
    },
    "Kerry": {
        "Monday": ("12:00", "21:00"),
        "Tuesday": ("09:00", "18:00"),
        "Wednesday": ("08:00", "17:00"),
        "Thursday": ("08:00", "17:00"),
        "Friday": ("08:00", "17:00")
    },
    "Cesar": {
        "Monday": ("12:00", "21:00"),
        "Tuesday": ("09:00", "18:00"),
        "Wednesday": ("09:00", "18:00"),
        "Thursday": ("08:00", "17:00"),
        "Friday": ("08:00", "17:00")
    },
    "Deepna": {
        "Monday": ("08:00", "17:00"),
        "Tuesday": ("08:00", "17:00"),
        "Wednesday": ("09:00", "18:00"),
        "Thursday": ("12:00", "21:00"),
        "Friday": ("08:00", "17:00")
    },
    "Sony": {
        "Tuesday": ("08:00", "14:00"),
        "Wednesday": ("08:00", "14:00"),
        "Thursday": ("15:00", "21:00"),
        "Friday": ("08:00", "14:00")
    },
    "Gabby": {
        "Monday": ("12:00", "21:00"),
        "Wednesday": ("09:00", "18:00"),
        "Friday": ("08:00", "17:00")
    },
    "Luna": {
        "Monday": ("08:00", "17:00"),
        "Tuesday": ("08:00", "17:00"),
        "Thursday": ("12:00", "21:00")
    },
    "Jaryn": {
        "Monday": ("08:00", "17:00"),
        "Tuesday": ("09:00", "18:00"),
        "Wednesday": ("08:00", "17:00"),
        "Thursday": ("17:00", "21:00")
    }
}

# Define library hours
library_hours = {
    "Monday": ("10:00", "21:00"),
    "Tuesday": ("10:00", "18:00"),
    "Wednesday": ("10:00", "18:00"),
    "Thursday": ("10:00", "21:00"),
    "Friday": ("10:00", "17:00")
}

# Convert time strings to datetime objects
def convert_to_datetime(time_str):
    return datetime.strptime(time_str, "%H:%M")

# Create a dictionary to store the hourly schedule for each desk
hourly_schedule = defaultdict(lambda: defaultdict(lambda: defaultdict(str)))

# Define desk types
desks = ["AV", "Info", "Ref"]

# Function to check if staff member is available during a specific time
def is_available(member, day, time):
    if member in working_hours and day in working_hours[member]:
        start_time, end_time = working_hours[member][day]
        # Define lunch break times
        lunch_breaks = {
            "08:00-17:00": "12:00",
            "09:00-18:00": "13:00",
            "12:00-21:00": "16:00"
        }
        for period, lunch_time in lunch_breaks.items():
            period_start, period_end = period.split('-')
            if start_time == period_start and end_time == period_end:
                if time == lunch_time:
                    return False
        return convert_to_datetime(start_time) <= convert_to_datetime(time) < convert_to_datetime(end_time)
    return True

# Function to check if staff member is scheduled for more than 2 consecutive hours
def is_consecutive_hours(member, day, time):
    time_dt = convert_to_datetime(time)
    for i in range(1, 3):
        prev_time = (time_dt - timedelta(hours=i)).strftime("%H:%M")
        if member not in hourly_schedule[day][prev_time].values():
            return False
    return True

# Function to populate the schedule
def populate_schedule(day, start_time, end_time):
    current_time = convert_to_datetime(start_time)
    end_time = convert_to_datetime(end_time)
    while current_time < end_time:
        for desk in desks:
            available_staff = [
                member for member, info in staff.items()
                if sum([1 for t, d in hourly_schedule[day].items() if d == desk and member in t]) < info["max_hours"]
                and is_available(member, day, current_time.strftime("%H:%M"))
                and all(hourly_schedule[day][current_time.strftime("%H:%M")].get(d) != member for d in desks)
                and not (is_consecutive_hours(member, day, current_time.strftime("%H:%M")) and current_time.hour < 17)
            ]
            if available_staff:
                staff_member = random.choice(available_staff)
                hourly_schedule[day][current_time.strftime("%H:%M")][desk] = staff_member
        current_time += timedelta(hours=1)

# Populate schedule for each day
for day, hours in library_hours.items():
    populate_schedule(day, hours[0], hours[1])

# Convert the schedule to a pandas DataFrame
schedule_data = []

for day, hours in library_hours.items():
    current_time = convert_to_datetime(hours[0])
    end_time = convert_to_datetime(hours[1])
    while current_time < end_time:
        time_str = current_time.strftime("%H:%M")
        row = {"Day": day, "Time": time_str}
        for desk in desks:
            row[desk] = hourly_schedule[day][time_str].get(desk, "No staff scheduled")
        schedule_data.append(row)
        current_time += timedelta(hours=1)

df = pd.DataFrame(schedule_data)

# Save the DataFrame to an Excel file
df.to_excel("desk_schedule.xlsx", index=False)

print("Schedule has been saved to 'desk_schedule.xlsx'")


Schedule has been saved to 'desk_schedule.xlsx'
